In [1]:
# Section must be included at the beginning of each new notebook. Remember to change the app name. 
# If you're using VirtualBox, change the below to '/home/user/spark-2.1.1-bin-hadoop2.7'
import findspark
findspark.init('/opt/spark')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('basics').getOrCreate()

In [2]:
# Let's read in the data. If you open the dataset, you'll find that each column has a header. We specify that by stating that header=True.
# To make our lives easier, we can also use 'inferSchema' when importing CSVs. This automatically detects data types.
# If you would like to manually change data types, refer to this article: https://medium.com/@mrpowers/adding-structtype-columns-to-spark-dataframes-b44125409803
df = spark.read.csv('Datasets/aggreated_data.csv',header=True,inferSchema=True)

In [3]:
df = df.na.drop()

# df = df.na.drop(subset="road_surface_conditions")
# df = df.na.drop(subset="light_conditions")
# df = df.na.drop(subset="weather_conditions")
# df = df.na.drop(subset="accident_severity")
# df = df.na.drop(subset="day_of_week")

# df = df.na.drop(subset="special_conditions_at_site")
# df = df.na.drop(subset="pedestrian_movement")
df = df.na.drop(subset="age_of_vehicle")
df = df.na.drop(subset="sex_of_driver")
df = df.na.drop(subset="age_of_driver")
df = df.na.drop(subset="junction_location")
df = df.na.drop(subset="junction_detail")
df = df.na.drop(subset="junction_control")
df = df.na.drop(subset="day_of_week")
df = df.na.drop(subset="accident_severity")



print("Total data points2:", df.count())

Total data points2: 285331


In [4]:
features = [
          'age_of_vehicle','sex_of_driver','age_of_driver','junction_location','junction_detail','junction_control','day_of_week','accident_severity']
# features_s = ['light_conditions','weather_conditions']


df1 = df.select(*features)
df1 = df1.filter(df1.accident_severity > 1)
df1 = df1.filter(df1.age_of_vehicle > 0)
df1 = df1.filter(df1.junction_control > 0)




In [5]:

df2 = df1.filter(df1.accident_severity == 2)
print(df2.count())



13862


In [6]:
df3 = df1.filter(df1.accident_severity == 3).limit(df2.count())
print(df3.count())


13862


In [7]:
df1 = df2.union(df3)
print(df1.count()) 

27724


In [8]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
# categoricalColumns = ['road_surface_conditions','light_conditions','weather_conditions']

ageofvehicle_indexer = StringIndexer(inputCol='age_of_vehicle',outputCol='ageofvehicleIndexer').setHandleInvalid("skip")
sexofdriver_indexer = StringIndexer(inputCol='sex_of_driver',outputCol='sexofdriverIndexer').setHandleInvalid("skip")
ageofdriver_indexer = StringIndexer(inputCol='age_of_driver',outputCol='ageofdriverIndexer').setHandleInvalid("skip")
junctionlocation_indexer = StringIndexer(inputCol='junction_location',outputCol='junctionlocationIndexer').setHandleInvalid("skip")
junctiondetail_indexer = StringIndexer(inputCol='junction_detail',outputCol='junctiondetailIndexer').setHandleInvalid("skip")
junctioncontrol_indexer = StringIndexer(inputCol='junction_control',outputCol='junctioncontrolIndexer').setHandleInvalid("skip")
dayofweek_indexer = StringIndexer(inputCol='day_of_week',outputCol='dayofweekIndexer').setHandleInvalid("skip")


accidentSeverity_indexer = StringIndexer(inputCol='accident_severity',outputCol='label').setHandleInvalid("skip")


# road_surface_conditions_encoder = OneHotEncoder(inputCol='road_surface_conditions_indexer',outputCol='road_surface_conditionsVec')
ageofvehicle_encoder = OneHotEncoder(inputCol='ageofvehicleIndexer',outputCol='ageofvehicleVec')
sexofdriver_encoder = OneHotEncoder(inputCol='sexofdriverIndexer',outputCol='sexofdriverVec')
ageofdriver_encoder = OneHotEncoder(inputCol='ageofdriverIndexer',outputCol='ageofdriverVec')
junctionlocation_encoder = OneHotEncoder(inputCol='junctionlocationIndexer',outputCol='junctionlocationVec')
junctiondetail_encoder = OneHotEncoder(inputCol='junctiondetailIndexer',outputCol='junctiondetailVec')
junctioncontrol_encoder = OneHotEncoder(inputCol='junctioncontrolIndexer',outputCol='junctioncontrolVec')
dayofweek_encoder = OneHotEncoder(inputCol='dayofweekIndexer',outputCol='dayofweekVec')


# And finally, using vector assembler to turn all of these columns into one column (named features).
assembler = VectorAssembler(inputCols=['ageofvehicleVec','sexofdriverVec','ageofdriverVec','junctionlocationVec','junctiondetailVec','junctioncontrolVec','dayofweekVec'], outputCol="features")

In [9]:
from pyspark.ml import Pipeline

# Then go through our steps. It's essentially sequential to the above.
pipeline = Pipeline(stages=[ageofvehicle_indexer, sexofdriver_indexer, ageofdriver_indexer,junctionlocation_indexer,junctiondetail_indexer,junctioncontrol_indexer,dayofweek_indexer,
                            accidentSeverity_indexer,
                            ageofvehicle_encoder, sexofdriver_encoder,ageofdriver_encoder,junctionlocation_encoder,junctiondetail_encoder,junctioncontrol_encoder,dayofweek_encoder,
                            assembler])

# Now that we've got a number of steps, let's apply it to the DataFrame.
pipeline_model = pipeline.fit(df1)

# Incorporate results into a new DataFrame.
pipe_df = pipeline_model.transform(df1)

# Remove all variables other than features and label. 
pipe_df = pipe_df.select('label', 'features')

In [10]:
pipe_df.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(164,[7,54,71,138...|
|  0.0|(164,[2,53,57,143...|
|  0.0|(164,[0,53,86,139...|
|  0.0|(164,[8,53,87,138...|
|  0.0|(164,[24,54,114,1...|
|  0.0|(164,[4,53,57,138...|
|  0.0|(164,[9,53,78,139...|
|  0.0|(164,[1,54,64,139...|
|  0.0|(164,[4,53,58,140...|
|  0.0|(164,[6,53,78,140...|
|  0.0|(164,[0,53,81,138...|
|  0.0|(164,[0,53,81,138...|
|  0.0|(164,[1,53,78,138...|
|  0.0|(164,[0,53,66,139...|
|  0.0|(164,[4,53,57,139...|
|  0.0|(164,[2,53,91,138...|
|  0.0|(164,[1,53,71,138...|
|  0.0|(164,[6,53,60,138...|
|  0.0|(164,[0,53,63,138...|
|  0.0|(164,[7,53,100,13...|
+-----+--------------------+
only showing top 20 rows



In [11]:
from pyspark.ml.classification import DecisionTreeClassifier

# Split our data. Note that the new DataFrame is being used.
train_data1, test_data1 = pipe_df.randomSplit([0.8,0.2])
#print("Training Dataset Count: " + str(train_data1.count()))
#print("Test Dataset Count: " + str(test_data1.count()))



In [12]:
dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'label', maxDepth = 3)
dtModel = dt.fit(train_data1)

predictions = dtModel.transform(test_data1)

#setHandleInvalid("skip")

In [13]:
predictions.show()

+-----+--------------------+---------------+--------------------+----------+
|label|            features|  rawPrediction|         probability|prediction|
+-----+--------------------+---------------+--------------------+----------+
|  0.0|(164,[0,53,56,138...|  [136.0,512.0]|[0.20987654320987...|       1.0|
|  0.0|(164,[0,53,56,138...|  [136.0,512.0]|[0.20987654320987...|       1.0|
|  0.0|(164,[0,53,56,139...|   [50.0,383.0]|[0.11547344110854...|       1.0|
|  0.0|(164,[0,53,56,140...|[5165.0,3237.0]|[0.61473458700309...|       0.0|
|  0.0|(164,[0,53,57,140...|[5165.0,3237.0]|[0.61473458700309...|       0.0|
|  0.0|(164,[0,53,58,138...|[3260.0,3946.0]|[0.45240077713016...|       1.0|
|  0.0|(164,[0,53,58,140...|[5165.0,3237.0]|[0.61473458700309...|       0.0|
|  0.0|(164,[0,53,59,144...|     [19.0,0.0]|           [1.0,0.0]|       0.0|
|  0.0|(164,[0,53,60,138...|[3260.0,3946.0]|[0.45240077713016...|       1.0|
|  0.0|(164,[0,53,60,138...|[3260.0,3946.0]|[0.45240077713016...|       1.0|

In [14]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator()
print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))

Test Area Under ROC: 0.49071218587008064
